### **基于TGARCH_M模型的股票型基金投资风格漂移动态识别及原因分析**

In [ ]:
import pandas as pd
from arch import arch_model, univariate
import datetime as dt
import numpy as np
import math
import time

from arch.unitroot import ADF
from statsmodels.tsa import stattools  # 白噪声检验:Ljung-Box检验
from statsmodels.tsa.arima.model import ARIMA #导入ARIMA模型
from rpy2.robjects import r
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from collections import defaultdict
pandas2ri.activate()
from rpy2.robjects import globalenv
importr("rugarch")

h:\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
h:\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


ValueError: r_home is None. Try python -m rpy2.situation

In [30]:
stata_data = pd.read_stata('./1.dta')

In [31]:
stata_data

,时间,基准收益率,基金收益率,date,residual,e1,varlist
0,2021-08-11,-0.005476,0.000000,0,-0.003018,0.003018,0.000020
1,2021-08-12,-0.008312,-0.009598,1,-0.007913,-0.001685,0.000021
2,2021-08-13,-0.005459,-0.003716,2,-0.003031,-0.000685,0.000020
3,2021-08-16,-0.000982,0.002974,3,-0.002104,0.005078,0.000021
4,2021-08-17,-0.020971,-0.025946,4,-0.015341,-0.010605,0.000020
...,...,...,...,...,...,...,...
236,2022-08-03,-0.009659,-0.002258,236,-0.007735,0.005477,0.000020
237,2022-08-04,0.008359,0.005860,237,0.006498,-0.000638,0.000020
238,2022-08-05,0.013232,0.015165,238,0.010339,0.004827,0.000020
239,2022-08-08,-0.002102,0.003094,239,-0.001792,0.004886,0.000020


In [116]:
fund_data = pd.read_excel('../Data/基金净值.xlsx', index_col=0, sheet_name=0)
index_data = pd.read_excel('../Data/指数净值.xlsx', index_col=0, sheet_name=0)
fund_info = pd.read_excel('../Data/基金净值.xlsx', index_col=0, sheet_name=1)
index_info = pd.read_excel('../Data/指数净值.xlsx', index_col=0, sheet_name=1)
fund2index = pd.read_excel('../Data/基金-指数对应关系.xlsx')

In [117]:
fund2index

,证券代码,证券名称,主要跟踪标的代码-第1名,标的指数简称-第1名
0,009127.OF,嘉实基础产业优选股票C,H30169.CSI,资本品
1,009126.OF,嘉实基础产业优选股票A,H30169.CSI,资本品
2,012181.OF,中银智能制造股票C,H11054.CSI,装备产业
3,001644.OF,汇丰晋信智造先锋股票C,H11054.CSI,装备产业
4,001643.OF,汇丰晋信智造先锋股票A,H11054.CSI,装备产业
...,...,...,...,...
815,010420.OF,民生加银成长优选股票,H30351.CSI,500成长
816,015001.OF,工银物流产业股票C,000957.SH,300运输
817,001718.OF,工银物流产业股票A,000957.SH,300运输
818,014635.OF,景顺长城ESG量化股票C,931463.CSI,300ESG


In [217]:
# 取交易日序列，交易日在基金净值表中表现稳定：
fund_data = fund_data.dropna(axis = 1)
trade_dates = fund_data.index

# 指数代码这块数据质量不佳，需要进行数据清洗，一个是去掉交易日，一个是找到全为非0的数据list
index_data = index_data.loc[trade_dates]
index_data = index_data[index_data.all()[index_data.all() == True].index]

In [218]:
fund2index

,证券代码,证券名称,主要跟踪标的代码-第1名,标的指数简称-第1名
0,009127.OF,嘉实基础产业优选股票C,H30169.CSI,资本品
1,009126.OF,嘉实基础产业优选股票A,H30169.CSI,资本品
2,012181.OF,中银智能制造股票C,H11054.CSI,装备产业
3,001644.OF,汇丰晋信智造先锋股票C,H11054.CSI,装备产业
4,001643.OF,汇丰晋信智造先锋股票A,H11054.CSI,装备产业
...,...,...,...,...
815,010420.OF,民生加银成长优选股票,H30351.CSI,500成长
816,015001.OF,工银物流产业股票C,000957.SH,300运输
817,001718.OF,工银物流产业股票A,000957.SH,300运输
818,014635.OF,景顺长城ESG量化股票C,931463.CSI,300ESG


In [222]:
fund_data_rtn = (np.log(fund_data) - np.log(fund_data.shift(1)))*100
fund_data_rtn = fund_data_rtn.dropna()
index_data_rtn = (np.log(index_data) - np.log(index_data.shift(1)))*100
index_data_rtn = index_data_rtn.dropna()

In [227]:
fund_data_rtn.apply(lambda x: np.median(x))

000008.OF    0.115291
000042.OF    0.025905
000051.OF   -0.052968
000059.OF   -0.037362
000082.OF    0.000000
               ...   
920002.OF    0.075527
920003.OF    0.109953
920922.OF    0.076190
920923.OF    0.108442
960000.OF   -0.035595
Length: 1494, dtype: float64

,000001.SH,000002.SH,000003.SH,000004.SH,000005.SH,000006.SH,000007.SH,000008.SH,000009.SH,000010.SH,...,H30214.CSI,H30215.CSI,H30216.CSI,H30217.CSI,H30218.CSI,H30219.CSI,H30220.CSI,H30221.CSI,H30222.CSI,H30223.CSI
2021-08-10,3529.9309,3699.6801,264.2945,3368.1006,2895.9057,5560.1435,4017.2788,2788.9345,6347.1670,10171.6895,...,3627.2064,2961.6669,1922.6020,19980.4797,1904.4500,3221.3055,4292.2894,3677.4868,2704.8035,1253.9461
2021-08-11,3532.6213,3702.5080,264.1320,3361.5049,2909.4427,5756.9497,4042.7029,2797.3695,6398.8183,10122.0361,...,3645.1487,2946.1615,1952.5287,19505.7654,1902.9568,3242.8656,4267.2332,3697.4960,2716.0670,1266.5564
2021-08-12,3524.7374,3694.2263,264.3699,3359.5199,2896.3600,5721.1474,4030.9433,2783.2258,6423.6104,10052.3393,...,3630.0925,2943.2700,2014.6109,19281.5342,1898.6278,3285.5554,4279.1761,3706.7069,2719.7189,1258.4334
2021-08-13,3516.2989,3685.3446,265.3974,3348.4591,2890.6381,5659.0505,4029.3131,2782.1107,6420.2052,10024.3615,...,3615.0509,2946.0737,2011.4656,19185.9893,1919.1844,3280.8692,4234.0232,3698.5936,2700.5507,1256.4189
2021-08-16,3517.3447,3686.4356,265.6988,3341.0213,2908.2473,5677.2169,4061.1858,2790.1151,6412.2524,10023.5172,...,3678.7129,2944.9444,1990.2238,19425.4025,1942.3150,3315.5903,4237.6585,3771.1100,2721.4956,1277.2494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-03,3163.6741,3314.7471,301.2736,3028.3641,2486.8292,5764.4661,4231.2171,2389.4723,5931.6330,8469.8327,...,3800.2663,3100.7393,2158.4148,13306.1406,1930.5392,2864.0891,3793.1114,3644.0381,2544.3108,1451.1383
2022-08-04,3189.0391,3341.3913,300.7329,3050.2133,2507.6515,5777.4266,4278.4359,2412.1266,5952.0286,8551.4386,...,3821.6385,3093.5560,2135.4211,13410.6111,1949.4080,2910.6807,3850.7774,3736.4693,2545.0972,1447.3483
2022-08-05,3227.0267,3381.2397,301.8829,3089.2898,2549.8837,5833.6119,4306.8435,2435.9853,6050.3396,8665.9626,...,3852.2817,3116.0701,2152.0429,13751.7371,1957.4588,2946.0591,3944.3048,3734.1863,2567.7456,1452.6165
2022-08-08,3236.9337,3391.3375,303.7149,3109.8456,2547.0318,5813.1825,4321.6609,2424.3063,6092.1164,8646.8136,...,3866.1986,3230.3487,2160.1236,13734.1319,1963.5552,2962.9561,3953.2467,3748.9615,2565.8060,1430.8527


000001.SH     True
000002.SH     True
000003.SH     True
000004.SH     True
000005.SH     True
              ... 
H30219.CSI    True
H30220.CSI    True
H30221.CSI    True
H30222.CSI    True
H30223.CSI    True
Length: 1193, dtype: bool

In [110]:
index_data.loc[time_range].all(axis = 1)

2021-08-10    False
2021-08-11    False
2021-08-12    False
2021-08-13    False
2021-08-16    False
              ...  
2022-08-04    False
2022-08-05    False
2022-08-08    False
2022-08-09    False
2022-08-10    False
Length: 245, dtype: bool

In [113]:
test = index_data.loc[time_range,'000001.SH']

In [115]:
test[test == 0]

2021-09-26    0.0
2022-04-24    0.0
Name: 000001.SH, dtype: float64